## 1. 필요 라이브러리 호출

In [1]:
import pandas as pd
import requests
import json

path = "/Users/kook/Desktop/"

## 2. 감성분석 api 호출 및 함수 정의

In [2]:
import requests
import json
import pandas as pd

def Sentiment_API(df, column_name):

    client_id = '5ex4s8ep3j'
    client_secret = 'FOPrm2MzgLrbVzVE0I0fs8oKDzwLEms5bZTdfQ6z'
    url = "https://naveropenapi.apigw.ntruss.com/sentiment-analysis/v1/analyze"

    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
        "Content-Type": "application/json"
    }

    sentiments = []

    for content in df[column_name]:
        # 950글자까지로 지정
        # API가 1000까지만 처리됨
        data = {
            "content": content[:950]  
        }
        response = requests.post(url, data=json.dumps(data), headers=headers)
        result = json.loads(response.text)

        highlights = []
        for sentence in result["sentences"]:
            highlights.append(sentence.get("highlights", []))

        sentiments.append({
            "label": result["document"]["sentiment"],
            "confidence": result["document"]["confidence"],
            "highlights": highlights
        })

    sentiments_df = pd.DataFrame(sentiments)
    df = pd.concat([df, sentiments_df], axis=1)
    
    return df

def extract_highlighted_text(df, title_column, highlights_column):
    highlighted_texts = []

    for idx, row in df.iterrows():
        title = row[title_column]
        highlights = row[highlights_column]
        extracted_texts = []

        for highlight_set in highlights:
            for highlight in highlight_set:
                offset = highlight['offset']
                length = highlight['length']
                extracted_texts.append(title[offset:offset + length])
        
        highlighted_texts.append(' '.join(extracted_texts))
    
    df['highlighted_texts'] = highlighted_texts
    return df

In [3]:
# 감성분석 데이터셋
df = pd.read_excel("제주도 에너지 뉴스기사 크롤링.xlsx")
df

,company,url,title,content
0,이데일리,https://n.news.naver.com/mnews/article/018/000...,"""뮤지컬 '어쩌면 해피엔딩' 브로드웨이 공연, 꿈만 같아요""",'윌휴' 콤비 박천휴·윌 애런슨의 K뮤지컬오는 10월 미국 뉴욕 벨라스코 극장 개...
1,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,[전국 주요 신문 톱뉴스](29일 조간),"▲ 경향신문 = 윤 대통령, 이첩 당일 이종섭에 3차례 전화 ▲ 국민일보 = 채상..."
2,세계일보,https://n.news.naver.com/mnews/article/022/000...,"제주·광주 “국립트라우마치유센터, 정부 지원 요구 공동 대응”","4·3-5·18 교두보, 민주평화·상생 맞손인권·산업·문화·인사교류 등 6개 과제..."
3,중앙일보,https://n.news.naver.com/mnews/article/025/000...,"미국 지켜주는 미사일?…1기당 163억, 비싸고 말 많은 SM-3 [이철재의 밀담]",말 많은 무기를 드디어 도입하기로 확정됐다. 해상 탄도미사일 요격 미사일인 SM-...
4,강원도민일보,https://n.news.naver.com/mnews/article/654/000...,"한호연·이철규 후보, 현정부 국정평가 놓고 치열한 공방",22대 총선 동해·태백·삼척·정선 후보자 토론회22대 총선 강원 동해·태백·삼척·...
...,...,...,...,...
390,아시아경제,https://n.news.naver.com/mnews/article/277/000...,"[배터리완전정복](32)ESS, 전기차 캐즘 극복할 돌파구될까",편집자주지금은 배터리 시대입니다. 휴대폰·노트북·전기자동차 등 거의 모든 곳에 배...
391,경기일보,https://n.news.naver.com/mnews/article/666/000...,"[영상] 김동연 ""김포 서울 편입 공약은 정치적 사기극""","서울, 기후카드 비협조 주장엔 “4자 협의 정면으로 깨는 것”북자도 법안 1호 제..."
392,한겨레,https://n.news.naver.com/mnews/article/028/000...,Q. 기후변화로 1월29일부터 ‘서울의 봄’이라구요?,기후변화 ‘쫌’ 아는 기자들 2081~2100년 서울의 봄 당겨진다는 예측계절 시...
393,뉴시스,https://n.news.naver.com/mnews/article/003/001...,제주포럼 현장서 ‘아세안 플러스 알파’ 협력강화 논의,"오영훈 지사, 필리핀 하원 대표단·주한 아랍대사단 면담경제·관광·에너지 등 여러 ..."


## 3. 감성분석

In [4]:
df_sentiment = Sentiment_API(df, 'title')

In [5]:
df_with_highlights = extract_highlighted_text(df_sentiment, 'title', 'highlights')

In [6]:
df_sentiment

,company,url,title,content,label,confidence,highlights,highlighted_texts
0,이데일리,https://n.news.naver.com/mnews/article/018/000...,"""뮤지컬 '어쩌면 해피엔딩' 브로드웨이 공연, 꿈만 같아요""",'윌휴' 콤비 박천휴·윌 애런슨의 K뮤지컬오는 10월 미국 뉴욕 벨라스코 극장 개...,positive,"{'negative': 0.00995304, 'positive': 99.947815...","[[{'offset': 26, 'length': 6}]]",꿈만 같아요
1,연합뉴스,https://n.news.naver.com/mnews/article/001/001...,[전국 주요 신문 톱뉴스](29일 조간),"▲ 경향신문 = 윤 대통령, 이첩 당일 이종섭에 3차례 전화 ▲ 국민일보 = 채상...",neutral,"{'negative': 0.019764842, 'positive': 0.018615...","[[{'offset': 0, 'length': 21}]]",[전국 주요 신문 톱뉴스](29일 조간
2,세계일보,https://n.news.naver.com/mnews/article/022/000...,"제주·광주 “국립트라우마치유센터, 정부 지원 요구 공동 대응”","4·3-5·18 교두보, 민주평화·상생 맞손인권·산업·문화·인사교류 등 6개 과제...",neutral,"{'negative': 0.12634115, 'positive': 0.1428990...","[[{'offset': 19, 'length': 14}]]",정부 지원 요구 공동 대응
3,중앙일보,https://n.news.naver.com/mnews/article/025/000...,"미국 지켜주는 미사일?…1기당 163억, 비싸고 말 많은 SM-3 [이철재의 밀담]",말 많은 무기를 드디어 도입하기로 확정됐다. 해상 탄도미사일 요격 미사일인 SM-...,negative,"{'negative': 74.96424, 'positive': 0.025858501...","[[{'offset': 0, 'length': 11}], [{'offset': 10...",미국 지켜주는 미사일 일?…1 국 지켜주는 미
4,강원도민일보,https://n.news.naver.com/mnews/article/654/000...,"한호연·이철규 후보, 현정부 국정평가 놓고 치열한 공방",22대 총선 동해·태백·삼척·정선 후보자 토론회22대 총선 강원 동해·태백·삼척·...,negative,"{'negative': 99.96666, 'positive': 0.004720351...","[[{'offset': 24, 'length': 6}]]",치열한 공방
...,...,...,...,...,...,...,...,...
390,아시아경제,https://n.news.naver.com/mnews/article/277/000...,"[배터리완전정복](32)ESS, 전기차 캐즘 극복할 돌파구될까",편집자주지금은 배터리 시대입니다. 휴대폰·노트북·전기자동차 등 거의 모든 곳에 배...,neutral,"{'negative': 0.068422526, 'positive': 0.034912...","[[{'offset': 18, 'length': 16}]]",전기차 캐즘 극복할 돌파구될까
391,경기일보,https://n.news.naver.com/mnews/article/666/000...,"[영상] 김동연 ""김포 서울 편입 공약은 정치적 사기극""","서울, 기후카드 비협조 주장엔 “4자 협의 정면으로 깨는 것”북자도 법안 1호 제...",negative,"{'negative': 99.98169, 'positive': 0.006518841...","[[{'offset': 10, 'length': 20}]]",김포 서울 편입 공약은 정치적 사기극
392,한겨레,https://n.news.naver.com/mnews/article/028/000...,Q. 기후변화로 1월29일부터 ‘서울의 봄’이라구요?,기후변화 ‘쫌’ 아는 기자들 2081~2100년 서울의 봄 당겨진다는 예측계절 시...,neutral,"{'negative': 0.018124063, 'positive': 0.019681...","[[{'offset': 3, 'length': 25}]]",기후변화로 1월29일부터 ‘서울의 봄’이라구요
393,뉴시스,https://n.news.naver.com/mnews/article/003/001...,제주포럼 현장서 ‘아세안 플러스 알파’ 협력강화 논의,"오영훈 지사, 필리핀 하원 대표단·주한 아랍대사단 면담경제·관광·에너지 등 여러 ...",neutral,"{'negative': 0.057513736, 'positive': 0.066708...","[[{'offset': 0, 'length': 29}]]",제주포럼 현장서 ‘아세안 플러스 알파’ 협력강화 논의


## 4. 감성분석 기여단어 추출(TF-IDF vectorizer 활용)

In [7]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer

# 형태소 분석기 초기화
okt = Okt()

# highlighted_texts 컬럼을 형태소 단위로 나누기
df_sentiment['tokenized_highlighted_texts'] = df_sentiment['highlighted_texts'].apply(lambda x: ' '.join(okt.nouns(x)))

# TF-IDF 벡터라이저 초기화
vectorizer = TfidfVectorizer()

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(df_sentiment['tokenized_highlighted_texts'])

# 단어 리스트 추출
terms = vectorizer.get_feature_names_out()

# 제목별로 중요한 단어 1개 추출
def extract_top_word(row):
    tfidf_vector = tfidf_matrix[row.name].toarray().flatten()
    top_index = tfidf_vector.argmax()
    top_word = terms[top_index]
    return top_word

# 각 제목별로 중요한 단어 1개 추출하여 'top_word' 컬럼에 저장
df_sentiment['top_word'] = df_sentiment.apply(extract_top_word, axis=1)

# 필요한 컬럼만 선택하여 최종 데이터프레임 생성
df_final = df_sentiment[['highlighted_texts', 'top_word', 'label', 'confidence']]

# 최종 데이터프레임 출력
print(df_final)

                 highlighted_texts top_word     label  \
0                           꿈만 같아요       가격  positive   
1            [전국 주요 신문 톱뉴스](29일 조간       신문   neutral   
2                   정부 지원 요구 공동 대응       요구   neutral   
3        미국 지켜주는 미사일 일?…1 국 지켜주는 미       미국  negative   
4                           치열한 공방       가격  negative   
..                             ...      ...       ...   
390               전기차 캐즘 극복할 돌파구될까      돌파구   neutral   
391           김포 서울 편입 공약은 정치적 사기극       공약  negative   
392      기후변화로 1월29일부터 ‘서울의 봄’이라구요     기후변화   neutral   
393  제주포럼 현장서 ‘아세안 플러스 알파’ 협력강화 논의      아세안   neutral   
394              원탁에 올린  사, 새 탄소흡수       원탁   neutral   

                                            confidence  
0    {'negative': 0.00995304, 'positive': 99.947815...  
1    {'negative': 0.019764842, 'positive': 0.018615...  
2    {'negative': 0.12634115, 'positive': 0.1428990...  
3    {'negative': 74.96424, 'positive': 0.025858501...  
4    {'negative': 99.96666, 'p

In [8]:
df_final.to_excel('제주도 에너지 감성분석 최고 기여 단어.xlsx', index=False)